1\. Write a function that converts number representation (bin<->dec<->hex)

In [18]:
# exercise solution
# implementing a class because it's easier to ask for a specific representation
# only works for positive integers


class Number:

    # look up tables for hexadecimal symbol conversion

    lookup_table_hex2bin = {
        "1" : "0001",
        "2" : "0010",
        "3" : "0011",
        "4" : "0100",
        "5" : "0101",
        "6" : "0110",
        "7" : "0111",
        "8" : "1000",
        "9" : "1001",
        "A" : "1010",
        "B" : "1011",
        "C" : "1100",
        "D" : "1101",
        "E" : "1110",
        "F" : "1111"
    }

    lookup_table_bin2hex = dict((v,k) for k,v in lookup_table_hex2bin.items())

    lookup_table_hex2dec = {
        '1' : 1,
        '2' : 2,
        '3' : 3,
        '4' : 4,
        '5' : 5,
        '6' : 6,
        '7' : 7,
        '8' : 8,
        '9' : 9,
        'A' : 10,
        'B' : 11,
        'C' : 12,
        'D' : 13,
        'E' : 14,
        'F' : 15
    }

    lookup_table_dec2hex = dict((v,k) for k,v in lookup_table_hex2dec.items())

    def __init__(self,num: str,typen: str):
        self.value = num
        self.type = typen

    def set_value(self,num: str,typen: str):
        self.value = num
        self.type = typen

    # binary representation 
    def binary(self):
        # binary to binary
        if self.type == "bin":
            print("Binary representation: {}".format(self.value))
        
        # decimal to binary
        elif self.type == "dec":
            binary_rep = ""
            temp_n = int(self.value)
            while temp_n > 0:
                binary_rep += str(temp_n % 2)
                temp_n = temp_n // 2
            
            print("The binary representation of {} is {}".format(self.value,binary_rep))

        # hexadecimal to binary
        elif self.type == "hex":
            binary_rep = ""
            for digit in self.value[::-1]:
                binary_rep = self.lookup_table_hex2bin[digit] + binary_rep
            print("The binary representation of {} is {}".format(self.value,binary_rep))
        else:
            print("Invalid representation type. Specify using set_value method.")
    
    # hexadecimal representation
    def hex(self):
        # binary to hexadecimal
        hex_rep = ""
        if self.type == "bin":
            
            #slicing up in groups of 4
            for n in range(0,len(self.value),4):
                # a 'nibble' is a group of 4 bits
                nibble=self.value[::-1][n:n+4][::-1]
                
                # correcting short values
                if len(nibble)<4:
                    nibble='0'*(4-len(nibble)%4)+nibble

                hex_rep=self.lookup_table_bin2hex[nibble]+hex_rep
            
            print("The hexadecimal representation of {} is {}".format(self.value,hex_rep))
        
        # decimal to hexadecimal
        elif self.type == "dec":
            temp_n = int(self.value)
            hex_rep = ""
            
            while temp_n > 0:
                hex_rep = str(self.lookup_table_dec2hex[temp_n%16]) + hex_rep
                temp_n = temp_n // 16

            print("The hexadecimal representation of {} is {}".format(self.value,hex_rep))

        else:
            print("Invalid representation type. Specify using set_value method.")

    def dec(self):
        # binary to decimal
        if self.type == "bin":
            decimal = 0
            for exp,digit in enumerate(self.value[::-1]):
                decimal += int(digit)*2**exp
            
            print("The decimal representation of {} is {}".format(self.value,decimal))

        # hexadecimal to decimal
        elif self.type == "hex":
            decimal = 0
            for exp,digit in enumerate(self.value[::-1]):
                decimal += int(self.lookup_table_hex2dec[digit])*16**exp

            print("The decimal representation of {} is {}".format(self.value,decimal))

        else:
            print("Invalid representation type. Specify using set_value method.")

                

# let's test this


num = Number("27","dec")
num.binary()
num.hex()
num.set_value("3A","hex")
num.binary()
num.dec()
num.set_value("1101101101100","bin")
num.hex()
num.dec()


The binary representation of 27 is 11011
The hexadecimal representation of 27 is 1B
The binary representation of 3A is 00111010
The decimal representation of 3A is 58
The hexadecimal representation of 1101101101100 is 1B6C
The decimal representation of 1101101101100 is 7020


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [ ]:
def word2float(num: str):
    

3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [ ]:
def deriv(f,x,delta):
    return (f(x+delta)-f(x))/delta

print("Derivative of x(x-1) evaluated at 1 using finite difference 0.01: \n",deriv(lambda x: x*(x-1),1,0.01))


for ex in [4,6,8,10,12,14]:
    d = float("1e-{}".format(ex))
    print(deriv(lambda x: x*(x-1),1,d))
    


Derivative of x(x-1) evaluated at 1 using finite difference 0.01: 
 1.010000000000001
1.0000999999998899
1.0000009999177333
1.0000000039225287
1.000000082840371
1.0000889005833413
0.9992007221626509


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 
